In [ ]:
# **U-Net — 기본 구조부터 학습 및 시각화까지**

 **U-Net 기반 이미지 세그멘테이션** 실습을 목표로 합니다.  
다운샘플(Encoder) → 업샘플(Decoder) + **Skip Connection** 구조가 **왜 분할(segmentation)에 강한지**를 직접 확인합니다.

## 학습 목표
- U-Net의 **Encoder/Decoder/Skip Connection** 흐름을 코드에서 찾을 수 있다.
- 입력/출력 텐서 shape이 단계별로 어떻게 바뀌는지 설명할 수 있다.
- 간단한 데이터셋으로 **학습 → 검증 → 예측 시각화**까지 한 번에 실행할 수 있다.
> Segmentation은 “어디에 무엇이 있는가”를 픽셀 단위로 예측합니다.  
> 그래서 U-Net은 **공간 정보(spatial detail)** 를 보존/복원하기 위한 구조적 장치를 적극적으로 사용합

In [ ]:
## 2) U-Net 모델 구현

- **목적:** U-Net의 표준 블록을 구현하고, Encoder/Decoder의 연결 구조를 코드로 확인합니다.
- **관찰 포인트**
  - `DoubleConv`(Conv–BN–ReLU 반복)이 feature 추출에 어떻게 쓰이는지
  - Down path에서 **Pooling/Stride**로 해상도가 줄어드는 지점
  - Up path에서 **Up-sampling(ConvTranspose2d 또는 bilinear)** 이 적용되는 지점
  - Skip 연결 시 **concat 채널 수**가 어떻게 증가하는지(가장 흔한 shape mismatch 원인)

> 구현을 따라가면서 각 단계의 feature map 크기(H×W)와 채널(C)을 메모해 두면 이해가 훨씬 빨라집니다.

In [ ]:
# =========================
# 3) U-Net 모델 정의
# =========================
# 이 섹션의 목표:
# - U-Net의 "인코더(Down) → 보틀넥 → 디코더(Up) + Skip Connection" 구조를
#   PyTorch 코드로 직접 따라가며 이해합니다.
#
# 핵심 아이디어(한 줄):
# - Down에서 공간 해상도(H,W)는 줄이고 채널(C)은 늘리면서 특징을 추출하고,
#   Up에서 해상도를 복원하면서 Down 단계의 특징맵을 Skip으로 concat하여
#   localization(위치 정보)을 되살립니다.

class DoubleConv(nn.Module):
    """U-Net 기본 블록: (Conv → ReLU) × 2

    - 첫 번째 Conv가 채널을 '중간 채널(mid_channels)'로 바꾸고,
      두 번째 Conv가 '출력 채널(out_channels)'로 맞춥니다.
    - 기본값(mid_channels=None)일 때는 mid_channels=out_channels로 두어,
      (C_in → C_out → C_out) 형태가 됩니다.

    입력/출력 텐서 형태:
      - 입력:  (B, C_in, H, W)
      - 출력:  (B, C_out, H, W)  # padding=1 이라 H,W 유지
    """
    def __init__(self, in_channels: int, out_channels: int, mid_channels: int | None = None):
        super().__init__()

        # U-Net에서 업샘플 후 concat을 하면 채널이 2배가 되므로,
        # bilinear 업샘플링을 쓸 때는 mid_channels=in_channels//2 처럼
        # '중간 채널'을 줄여주는 방식이 흔히 사용됩니다.
        if mid_channels is None:
            mid_channels = out_channels

        self.net = nn.Sequential(
            # 1) (C_in → C_mid)
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.ReLU(inplace=True),

            # 2) (C_mid → C_out)
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.ReLU(inplace=True),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)


In [ ]:
class Down(nn.Module):
    """Downsampling 블록: MaxPool(2)로 해상도 1/2 → DoubleConv

    입력/출력 텐서 형태:
      - 입력:  (B, C_in,  H,  W)
      - 출력:  (B, C_out, H/2, W/2)
    """
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # 1) 해상도 축소
        x = self.pool(x)           # (B, C_in, H/2, W/2)
        # 2) 채널 확장 + 특징 추출
        x = self.conv(x)           # (B, C_out, H/2, W/2)
        return x



In [ ]:
class Up(nn.Module):
    """Upsampling 블록: 업샘플링 → (Skip concat) → DoubleConv

    구현 관점(중요):
    - in_channels 는 concat 이후 채널 수를 의미합니다.
      예) x1(디코더) 채널=512, x2(skip) 채널=512 → concat 채널=1024 → in_channels=1024

    - bilinear=True:
        1) 업샘플은 파라미터 없는 bilinear interpolation으로 수행
        2) concat 후 DoubleConv에서 mid_channels를 in_channels//2 로 두어
           채널을 자연스럽게 '절반'으로 줄이는 방식(원 논문/레퍼런스 구현과 동일 계열)

    - bilinear=False:
        ConvTranspose2d로 업샘플 자체를 학습(파라미터 증가)
    """
    def __init__(self, in_channels: int, out_channels: int, bilinear: bool = False):
        super().__init__()

        if bilinear:
            # (B, C, H/2, W/2) → (B, C, H, W)
            self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)

            # bilinear일 때는 업샘플 후 채널 수(C)가 그대로 유지됩니다.
            # concat 결과(in_channels)를 DoubleConv로 처리하되,
            # 첫 conv의 출력(mid_channels)을 in_channels//2로 두어 채널을 줄입니다.
            self.conv = DoubleConv(in_channels, out_channels, mid_channels=in_channels // 2)
        else:
            # (B, C, H/2, W/2) → (B, C/2, H, W)  (deconv가 채널도 절반으로 줄여줌)
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1: torch.Tensor, x2: torch.Tensor) -> torch.Tensor:
        # x1: (B, C_dec, H/2, W/2)  / x2: (B, C_skip, H, W)

        # 1) 업샘플링: 해상도를 skip과 맞추기
        x1 = self.up(x1)  # bilinear: 채널 유지 / deconv: 채널이 절반으로 감소

        # 2) (필요 시) 패딩으로 크기 정렬
        #    - 홀수 크기 입력 등으로 인해 x1과 x2의 H/W가 1~2 픽셀 정도 다를 수 있습니다.
        diff_y = x2.size(2) - x1.size(2)
        diff_x = x2.size(3) - x1.size(3)
        x1 = F.pad(x1, [diff_x // 2, diff_x - diff_x // 2,  # left, right
                        diff_y // 2, diff_y - diff_y // 2]) # top, bottom

        # 3) 채널 방향 concat (skip 연결)
        #    cat dim=1 은 채널(C) 축
        x = torch.cat([x2, x1], dim=1)  # (B, C_skip + C_up, H, W) == (B, in_channels, H, W)

        # 4) conv 블록으로 특징 정제 + 채널 축소
        return self.conv(x)

In [ ]:
class OutConv(nn.Module):
    """마지막 1x1 conv: 채널을 클래스 수로 매핑

    예)
      - binary segmentation: n_classes=1 (logits 1채널)
      - multi-class:         n_classes=K (logits K채널)
    """
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.conv(x)

In [ ]:
class UNet(nn.Module):
    """U-Net 전체 모델

    전형적인 채널 구성 예:
      1 → 64 → 128 → 256 → 512 → 1024 (down)
      1024 → 512 → 256 → 128 → 64 (up)
      숫자 외워야 할 듯.
    """
    def __init__(self, n_channels: int, n_classes: int, bilinear: bool = False):
        super().__init__()

        # -------------------------
        # Encoder (Contracting path)
        # -------------------------
        self.inc = DoubleConv(n_channels, 64)   # (B, n_channels, H, W) → (B, 64, H, W)
        self.down1 = Down(64, 128)              # → (B, 128, H/2, W/2)
        self.down2 = Down(128, 256)             # → (B, 256, H/4, W/4)
        self.down3 = Down(256, 512)             # → (B, 512, H/8, W/8)

        # bilinear 업샘플링이면 파라미터/연산을 줄이기 위해 bottleneck 채널을 1024 대신 512로 줄이는 경우가 많음
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)  # → (B, 1024/f, H/16, W/16)

        # -------------------------
        # Decoder (Expanding path)
        # -------------------------
        # Up 블록의 in_channels는 "concat 후 채널" 기준으로 설계되어야 합니다.
        self.up1 = Up(1024, 512 // factor, bilinear)  # (skip=512)와 concat을 고려
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)

        # -------------------------
        # Output head
        # -------------------------
        self.outc = OutConv(64, n_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # -------------------------
        # Encoder: skip을 위해 각 단계 출력 저장
        # -------------------------
        x1 = self.inc(x)       # (B, 64, H, W)
        x2 = self.down1(x1)    # (B, 128, H/2, W/2)
        x3 = self.down2(x2)    # (B, 256, H/4, W/4)
        x4 = self.down3(x3)    # (B, 512, H/8, W/8)
        x5 = self.down4(x4)    # (B, 1024/f, H/16, W/16)

        # -------------------------
        # Decoder: 업샘플 + skip concat
        # -------------------------
        x = self.up1(x5, x4)   # (B, 512/f, H/8, W/8)
        x = self.up2(x, x3)    # (B, 256/f, H/4, W/4)
        x = self.up3(x, x2)    # (B, 128/f, H/2, W/2)
        x = self.up4(x, x1)    # (B, 64, H, W)

        # -------------------------
        # 최종 logits 출력
        # -------------------------
        logits = self.outc(x)  # (B, n_classes, H, W)
        return logits



In [ ]:
"""
    - Binary segmentation이면 보통 `BCEWithLogitsLoss`를 많이 사용(로짓 입력 주의)
    - Multi-class segmentation이면 `CrossEntropyLoss`(타깃은 class index) 사용
    - `sigmoid/softmax`를 **언제 적용해야 하는지**(loss와의 궁합)
  """
# 손실 함수 및 옵티마이저 설정
criterion = nn.BCEWithLogitsLoss()  # Sigmoid + Binary Cross Entropy
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# (옵션) 학습 스케줄러
# 20 에포크(Epoch)마다 학습률을 변경, 기존 학습률에 0.5를 곱합
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
